In [ ]:
from matplotlib import pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.animation as anim
from IPython.core.display import HTML
from mpl_toolkits import mplot3d
plt.rcParams["figure.autolayout"] = True
import scipy.integrate as integrate

source: https://quantumphysicssimulations.blogspot.com/2020/12/time-evolution-of-quantum-particle-in.html?fbclid=IwAR2YLkbsD1uabSGj74c1yVbD5usZD96t2UrsvFJ2t30kaYLarVDvasYLWIY

In [ ]:
plt.rcParams["figure.figsize"] = [7.50, 3.50]
plt.rcParams["figure.autolayout"] = True
plt.rcParams['animation.embed_limit'] = 2**128
plt.rcParams['text.usetex'] = False

L = 10 #wybrane
h_cross = 1 
m = 1
xpos = np.linspace(0, L, resolution)
ypos = np.linspace(0, L, resolution)

A = 0.4 
sigma = 0.1
k0 = 1
x0 = L/2 #wybrane
y0 = L/2 #wybrane

resolution = 50
frn = 300
fps = 250

cx = np.zeros(30, dtype = np.complex_)
cy = np.zeros(30, dtype = np.complex_)

def psi_nx(x, nx):
    # stany stacjonarne niezależne
    if(x >= 0 and x <= L):
        return np.sqrt(2/L)*np.sin(nx*np.pi*x/L)
    else:
        return 0

def psi_ny(y, ny):
    # stany stacjonarne niezależne
    if(y >= 0 and y <= L):
        return np.sqrt(2/L)*np.sin(ny*np.pi*y/L)
    else:
        return 0
    
for i in range(30):
    nx = i + 1
    I_Real = lambda x: psi_nx(x, nx)*np.real(psi0(x))
    I_Imag = lambda x: psi_nx(x, nx)*np.imag(psi0(x))
    cx[i] = complex(integrate.quad(I_Real, 0, L)[0],integrate.quad(I_Imag, 0, L)[0])

for i in range(30):
    ny = i + 1
    I_Real = lambda y: psi_ny(y, ny)*np.real(psi0(y))
    I_Imag = lambda y: psi_ny(y, ny)*np.imag(psi0(y))
    cy[i] = complex(integrate.quad(I_Real, 0, L)[0],integrate.quad(I_Imag, 0, L)[0])

def psi0(x):
    if(x <= L and x >= 0):
        return A*np.exp(-((x-xo)**2)/2*sigma)*np.exp(complex(0,k0*x))
    else:
        return 0

def psi0(y):
    if(y <= L and y >= 0):
        return A*np.exp(-((y-y0)**2)/20)*np.exp(complex(0,k0*y))
    else:
        return 0
    


def PSI_X(x, t):
    val = 0
    for i in range(30):
        nx = i+1
        val += cx[i]*psi_nx(x, nx)*np.exp(complex(0,-(nx**2)*(np.pi**2)*h_cross*t/(2*m*(L**2))))

    return val

def PSI_Y(y, t):
    val = 0
    for i in range(30):
        ny = i+1
        val += cy[i]*psi_ny(y, ny)*np.exp(complex(0,-(ny**2)*(np.pi**2)*h_cross*t/(2*m*(L**2))))

    return val

def sum_PSI(t):
    val = 0
    PSI_compl = []

    for i in range(0, 50):
        PSI_compl.append([])
        for j in range(0, 50):
            PSI_compl[i].append(np.abs(PSI_X(xpos[i], t)*PSI_Y(ypos[j], t))**2)
    #print(PSI_compl)
    return PSI_compl

x = np.linspace(0,L,resolution)
y = np.linspace(0,L,resolution)
X, Y = np.meshgrid(x, y)
Z = np.zeros((resolution, resolution, frn))
for i in range(frn):
    Z[:,:, i] = sum_PSI(0.01*i)

def change_plot(frame_number, Z, plot):
    plot[0].remove()
    plot[0] = ax.plot_surface(X, Y, Z[:, :, frame_number], cmap="magma")

fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(projection='3d')
#ax.text2D(0.42, 0.67, r"$\mathrm{\psi_{n_{x,y}}(x,y,t)=\sqrt{\frac{4}{L_xL_y}}sin\left(\frac{n_x\pi}{L_x}x\right)sin\left(\frac{n_y\pi}{L_y}y\right)\cdot e^{-i\frac{E_{x,y}}{\hbar}t}}$",
         # transform=ax.transAxes, fontsize = 13, rotation = -15, color = 'grey').set_zorder(1)
#ax.text2D(0.4, 0.63, "  $n_x={}$\n   $n_y={}$\n     $C={}^T$".format(nx,ny,C),
          #transform=ax.transAxes, fontsize = 14, rotation = -15, color = 'grey').set_zorder(1)
ax.set_zlim(np.min(Z),np.max(Z))
ax.set_xlabel('x', fontsize = 16)
ax.set_ylabel('y', fontsize = 16)
ax.set_zlabel('$\mid\Psi(\overline{r},t)\mid^2$', fontsize = 16)
#fig.suptitle(r"$\mid \Psi(\overline{r},t)\mid^2$", fontsize = 25)
plot = [ax.plot_surface(X, Y, Z[:,:,0], color='blue')]


ani = anim.FuncAnimation(fig, change_plot, frn, fargs=(Z, plot), interval=1000/fps)
writervideo = anim.FFMpegWriter(fps=60)
ani.save('2dwell_gauss.mp4', writer=writervideo)
HTML(ani.to_jshtml())